In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn.linear_model import LinearRegression
from scipy import stats
import pandas as pd
import collections
import arviz as az
import pystan
import pickle
import os

In [3]:
def gplvm_predictions(Q, kernel, path, data):
    for Q in Qs:
        for kernel in kernels:
            samples = pd.read_csv(path+'samples/s_Q{}_kernel{}.csv'.format(Q, kernel), sep=',', comment='#')

            ## COVARIANCE MATRIX ##
            K_ = samples.filter(regex=("^K"))
            K_ = K_.replace([np.inf, -np.inf], np.nan).dropna(0).mean(axis=0)
            K_mat = K_.to_numpy().reshape(N,N)
            K = pd.DataFrame(K_mat)
            K.to_csv(path+'samples/K_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## FUNCTION NOISE ##
            noise_std_ = samples.filter(regex=("^noise_std"))
            noise_std_.replace([np.inf, -np.inf], np.nan).dropna(0)
            noise_std_np = noise_std_.mean(axis=0)
            noise_std = pd.DataFrame(noise_std_np)
            noise_std.to_csv(path+'samples/noise_std_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## KERNEL NOISE ##
            kernel_std__ = samples.filter(regex=("^diag_stds"))
            kernel_std__.replace([np.inf, -np.inf], np.nan).dropna(0)
            kernel_std_ = kernel_std__.mean(axis=0)
            kernel_std_np = kernel_std_.to_numpy().reshape(N)
            kernel_std = pd.DataFrame(kernel_std_np)
            kernel_std = kernel_std.rename_axis('D')
            kernel_std = kernel_std.rename_axis('N', axis='columns')
            kernel_std.to_csv(path+'samples/kernel_std_Q{}_kernel{}.csv'.format(Q, kernel), index=False)
            
            ## LATENT SPACE POSITIONS ##
            X__ = samples.filter(regex=("^X"))
            X__.replace([np.inf, -np.inf], np.nan).dropna(0)
            X_ = X__.mean(axis=0)
            X_np = X_.to_numpy().reshape(Q,N)
            X = pd.DataFrame(X_np)
            X = X.rename_axis('Latent Dimension')
            X = X.rename_axis('Stock', axis='columns')
            X.to_csv(path+'samples/X_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## PREDICTIONS ##
            K_noise = np.add(K, np.matmul(np.eye(N), noise_std))
            prod1 = np.matmul(np.linalg.pinv(K_noise.to_numpy()),data)
            predictions_ = np.matmul(K.values, prod1)
            predictions = pd.DataFrame(predictions_)
            predictions = predictions.rename_axis('D')
            predictions = predictions.rename_axis('N', axis='columns')
            predictions.to_csv(path+'samples/Y_hat-Q{}-kernel{}.csv'.format(Q, kernel))
            model_analysis(Q, kernel, path, data, predictions, samples)
    return K, noise_std, kernel_std, X, predictions

In [4]:
def studt_predictions(Qs, kernels, paths, data):
    for Q in Qs:
        for kernel in kernels:
            samples = pd.read_csv(path+'samples/s_Q{}_kernel{}.csv'.format(Q, kernel), sep=',', comment='#')

            ## COVARIANCE MATRIX ##
            K_ = samples.filter(regex=("^K"))
            K_ = K_.replace([np.inf, -np.inf], np.nan).dropna(0).mean(axis=0)
            K_mat = K_.to_numpy().reshape(N,N)
            K = pd.DataFrame(K_mat)
            K.to_csv(path+'samples/K_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## FUNCTION NOISE ##
            noise_std_ = samples.filter(regex=("^noise_std"))
            noise_std_.replace([np.inf, -np.inf], np.nan).dropna(0)
            noise_std_np = noise_std_.mean(axis=0)
            noise_std = pd.DataFrame(noise_std_np)
            noise_std.to_csv(path+'samples/noise_std_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## KERNEL NOISE ##
            kernel_std__ = samples.filter(regex=("^diag_stds"))
            kernel_std__.replace([np.inf, -np.inf], np.nan).dropna(0)
            kernel_std_ = kernel_std__.mean(axis=0)
            kernel_std_np = kernel_std_.to_numpy().reshape(N)
            kernel_std = pd.DataFrame(kernel_std_np)
            kernel_std = kernel_std.rename_axis('D')
            kernel_std = kernel_std.rename_axis('N', axis='columns')
            kernel_std.to_csv(path+'samples/kernel_std_Q{}_kernel{}.csv'.format(Q, kernel), index=False)
            
            ## LATENT SPACE POSITIONS ##
            X__ = samples.filter(regex=("^X"))
            X__.replace([np.inf, -np.inf], np.nan).dropna(0)
            X_ = X__.mean(axis=0)
            X_np = X_.to_numpy().reshape(Q,N)
            X = pd.DataFrame(X_np)
            X = X.rename_axis('Latent Dimension')
            X = X.rename_axis('Stock', axis='columns')
            X.to_csv(path+'samples/X_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## PREDICTIONS ##
            K_noise = np.add(K, np.matmul(np.eye(N), noise_std))
            prod1 = np.matmul(np.linalg.pinv(K_noise.to_numpy()),data)
            predictions_ = np.matmul(K.values, prod1)
            predictions = pd.DataFrame(predictions_)
            predictions = predictions.rename_axis('D')
            predictions = predictions.rename_axis('N', axis='columns')
            predictions.to_csv(path+'samples/Y_hat-Q{}-kernel{}.csv'.format(Q, kernel))
            model_analysis(Q, kernel, path, data, predictions, samples)
            
            ## DEGREE OF FREEDOM ##
            t_nu__ = samples.filter(regex=("^t_nu"))
            t_nu__.replace([np.inf, -np.inf], np.nan).dropna(0)
            t_nu_ = t_nu__.mean(axis=0)
            t_nu = pd.DataFrame(t_nu_)
            t_nu.to_csv(path+'samples/kernel_std_Q{}_kernel{}.csv'.format(Q, kernel), index=False)
    return K, noise_std, kernel_std, kernel_std, t_nu, X, predictions

In [5]:
def vola_predictions(Qs, kernels, paths, data):
    for Q in Qs:
        for kernel in kernels:
            samples = pd.read_csv(path+'samples/s_Q{}_kernel{}.csv'.format(Q, kernel), sep=',', comment='#')

            ## COVARIANCE MATRIX ##
            K__ = samples.filter(regex=("^K"))
            K_ = K__.replace([np.inf, -np.inf], np.nan).dropna(0).mean(axis=0)
            K_mat = K_.to_numpy().reshape((D,N,N), order='F')
            K = pd.DataFrame(K_mat)
            #K_.to_csv(path+'samples/K_Q{}_kernel{}.csv'.format(Q, kernel))

            ## INTRINSIC NOISE ##
            noise_std_ = samples.filter(regex=("^noise_std"))
            noise_std_.replace([np.inf, -np.inf], np.nan).dropna(0)
            noise_std_np = noise_std_.mean(axis=0)
            noise_std = pd.DataFrame(noise_std_np)
            noise_std.to_csv(path+'samples/noise_std_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## LATENT SPACE POSITIONS ##
            X__ = samples.filter(regex=("^X"))
            X__.replace([np.inf, -np.inf], np.nan).dropna(0)
            X_ = X__.mean(axis=0)
            X_np = X_.to_numpy().reshape(Q,N)
            X = pd.DataFrame(X_np)
            X = X.rename_axis('Latent Dimension')
            X = X.rename_axis('Stock', axis='columns')
            X.to_csv(path+'samples/X_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## VOLATILITIES ##
            Sigma__ = samples.filter(regex=("^Sigma"))
            Sigma__.replace([np.inf, -np.inf], np.nan).dropna(0)
            Sigma_ = Sigma__.mean(axis=0)
            Sigma_np = Sigma_.to_numpy().reshape(D,N)
            Sigma = pd.DataFrame(Sigma_np)
            Sigma = Sigma.rename_axis('D')
            Sigma = Sigma.rename_axis('N', axis='columns')
            Sigma.to_csv(path+'samples/Sigma_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## PREDICTIONS ##
            predictions = pd.DataFrame()
            for d in range(-1,D-1):
                K_noise = np.add(pd.DataFrame(K_mat[d,:,:]), np.matmul(np.eye(N), noise_std_np))
                prod1 = np.matmul(np.linalg.pinv(K_noise),data)
                predictions___ = pd.DataFrame(np.matmul(K_mat[d,:,:], pd.DataFrame(prod1)))
                predictions__ = pd.DataFrame(predictions___.iloc[:,d])
                predictions = pd.concat([predictions,predictions__], axis=1, ignore_index=True)
            predictions.to_csv(path+'samples/Y_hat-Q{}-kernel{}.csv'.format(Q, kernel))
            model_analysis(Q, kernel, path, data, predictions, samples)
    return K, noise_std, X, Sigma, predictions

In [6]:
def time_predictions(Qs, kernels, paths, data):
    for Q in Qs:
        for kernel in kernels:
            samples = pd.read_csv(path+'samples/s_Q{}_kernel{}.csv'.format(Q, kernel), sep=',', comment='#')

            ## COVARIANCE MATRIX ##
            K_y__ = samples.filter(regex=("^K_y"))
            K_y_ = K_y__.replace([np.inf, -np.inf], np.nan).dropna(0).mean(axis=0)
            K_y_mat = K_y_.to_numpy().reshape((D,N,N), order='F')
            K_y_.to_csv(path+'samples/K_y_Q{}_kernel{}.csv'.format(Q, kernel))

            ## INTRINSIC NOISE ##
            noise_std_x_ = samples.filter(regex=("^noise_std_x"))
            noise_std_x_.replace([np.inf, -np.inf], np.nan).dropna(0)
            noise_std_x_np = noise_std_x_.mean(axis=0)
            noise_std_x = pd.DataFrame(noise_std_x_np)
            noise_std_x.to_csv(path+'noise_std_x_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            noise_std_y_ = samples.filter(regex=("^noise_std_y"))
            noise_std_y_.replace([np.inf, -np.inf], np.nan).dropna(0)
            noise_std_y_np = noise_std_y_.mean(axis=0)
            noise_std_y = pd.DataFrame(noise_std_y_np)
            noise_std_y.to_csv(path+'noise_std_y_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## KERNEL NOISE ##
            kernel_std_x__ = samples.filter(regex=("^kernel_std_x"))
            kernel_std_x__.replace([np.inf, -np.inf], np.nan).dropna(0)
            kernel_std_x_ = kernel_std_x__.mean(axis=0)
            kernel_std_x_np = kernel_std_x_.to_numpy()
            kernel_std_x = pd.DataFrame(kernel_std_x_np)
            kernel_std_x = kernel_std_x.rename_axis('D')
            kernel_std_x = kernel_std_x.rename_axis('N', axis='columns')
            kernel_std_x.to_csv(path+'kernel_std_x_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            kernel_std_y__ = samples.filter(regex=("^kernel_std_y"))
            kernel_std_y__.replace([np.inf, -np.inf], np.nan).dropna(0)
            kernel_std_y_ = kernel_std_y__.mean(axis=0)
            kernel_std_y_np = kernel_std_y_.to_numpy()
            kernel_std_y = pd.DataFrame(kernel_std_x_np)
            kernel_std_y = kernel_std_y.rename_axis('D')
            kernel_std_y = kernel_std_y.rename_axis('N', axis='columns')
            kernel_std_y.to_csv(path+'kernel_std_y_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## LATENT SPACE POSITIONS ##
            X__ = samples.filter(regex=("^X"))
            X__.replace([np.inf, -np.inf], np.nan).dropna(0)
            X_ = X__.mean(axis=0)
            prod = N*D
            X_temp = pd.DataFrame()
            for q in range(0,Q):
                X_np_temp = X_.iloc[(q*prod):((q+1)*prod)].to_numpy().reshape(N,D)
                X_temp_ = pd.DataFrame(X_np_temp)
                X_temp_ = X_temp_.rename_axis(f'Q{q+1}')
                X_temp = pd.concat([X_temp, X_temp_])
            X = X_temp.rename_axis('Stock No.')
            X = X.rename_axis('Days', axis='columns')
            X.to_csv(path+'X_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## PREDICTIONS ##
            predictions = pd.DataFrame()
            for d in range(-1,D-1):
                K_noise = np.add(pd.DataFrame(K_y_mat[d,:,:]), np.matmul(np.eye(N), noise_std_y_np))
                prod1 = np.matmul(np.linalg.pinv(K_noise),data)
                predictions___ = pd.DataFrame(np.matmul(K_y_mat[d,:,:], pd.DataFrame(prod1)))
                predictions__ = pd.DataFrame(predictions___.iloc[:,d])
                predictions = pd.concat([predictions,predictions__], axis=1, ignore_index=True)
            predictions.to_csv(path+'samples/Y_hat-Q{}-kernel{}.csv'.format(Q, kernel))
            model_analysis(Q, kernel, path, data, predictions, samples)
    return K_y, noise_std_x, noise_std_y, X, kernel_std_x, kernel_std_y, predictions

In [7]:
def timet_predictions(Qs, kernels, paths, data):
    for Q in Qs:
        for kernel in kernels:
            samples = pd.read_csv(path+'samples/s_Q{}_kernel{}.csv'.format(Q, kernel), sep=',', comment='#')

            ## COVARIANCE MATRIX ##
            K_y__ = samples.filter(regex=("^K_y"))
            K_y_ = K_y__.replace([np.inf, -np.inf], np.nan).dropna(0).mean(axis=0)
            K_y_mat = K_y_.to_numpy().reshape((D,N,N), order='F')
            K_y_.to_csv(path+'samples/K_y_Q{}_kernel{}.csv'.format(Q, kernel))

            ## INTRINSIC NOISE ##
            noise_std_x_ = samples.filter(regex=("^noise_std_x"))
            noise_std_x_.replace([np.inf, -np.inf], np.nan).dropna(0)
            noise_std_x_np = noise_std_x_.mean(axis=0)
            noise_std_x = pd.DataFrame(noise_std_x_np)
            noise_std_x.to_csv(path+'noise_std_x_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            noise_std_y_ = samples.filter(regex=("^noise_std_y"))
            noise_std_y_.replace([np.inf, -np.inf], np.nan).dropna(0)
            noise_std_y_np = noise_std_y_.mean(axis=0)
            noise_std_y = pd.DataFrame(noise_std_y_np)
            noise_std_y.to_csv(path+'noise_std_y_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## KERNEL NOISE ##
            kernel_std_x__ = samples.filter(regex=("^kernel_std_x"))
            kernel_std_x__.replace([np.inf, -np.inf], np.nan).dropna(0)
            kernel_std_x_ = kernel_std_x__.mean(axis=0)
            kernel_std_x_np = kernel_std_x_.to_numpy()
            kernel_std_x = pd.DataFrame(kernel_std_x_np)
            kernel_std_x = kernel_std_x.rename_axis('D')
            kernel_std_x = kernel_std_x.rename_axis('N', axis='columns')
            kernel_std_x.to_csv(path+'kernel_std_x_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            kernel_std_y__ = samples.filter(regex=("^kernel_std_y"))
            kernel_std_y__.replace([np.inf, -np.inf], np.nan).dropna(0)
            kernel_std_y_ = kernel_std_y__.mean(axis=0)
            kernel_std_y_np = kernel_std_y_.to_numpy()
            kernel_std_y = pd.DataFrame(kernel_std_x_np)
            kernel_std_y = kernel_std_y.rename_axis('D')
            kernel_std_y = kernel_std_y.rename_axis('N', axis='columns')
            kernel_std_y.to_csv(path+'kernel_std_y_Q{}_kernel{}.csv'.format(Q, kernel), index=False)
            
            ## DEGREE OF FREEDOM ##
            t_nu__ = samples.filter(regex=("^t_nu"))
            t_nu__.replace([np.inf, -np.inf], np.nan).dropna(0)
            t_nu_ = t_nu__.mean(axis=0)
            t_nu = pd.DataFrame(t_nu_)
            t_nu.to_csv(path+'samples/kernel_std_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## LATENT SPACE POSITIONS ##
            X__ = samples.filter(regex=("^X"))
            X__.replace([np.inf, -np.inf], np.nan).dropna(0)
            X_ = X__.mean(axis=0)
            prod = N*D
            X_temp = pd.DataFrame()
            for q in range(0,Q):
                X_np_temp = X_.iloc[(q*prod):((q+1)*prod)].to_numpy().reshape(N,D)
                X_temp_ = pd.DataFrame(X_np_temp)
                X_temp_ = X_temp_.rename_axis(f'Q{q+1}')
                X_temp = pd.concat([X_temp, X_temp_])
            X = X_temp.rename_axis('Stock No.')
            X = X.rename_axis('Days', axis='columns')
            X.to_csv(path+'X_Q{}_kernel{}.csv'.format(Q, kernel), index=False)

            ## PREDICTIONS ##
            predictions = pd.DataFrame()
            for d in range(-1,D-1):
                K_noise = np.add(pd.DataFrame(K_y_mat[d,:,:]), np.matmul(np.eye(N), noise_std_y_np))
                prod1 = np.matmul(np.linalg.pinv(K_noise),data)
                predictions___ = pd.DataFrame(np.matmul(K_y_mat[d,:,:], pd.DataFrame(prod1)))
                predictions__ = pd.DataFrame(predictions___.iloc[:,d])
                predictions = pd.concat([predictions,predictions__], axis=1, ignore_index=True)
            predictions.to_csv(path+'samples/Y_hat-Q{}-kernel{}.csv'.format(Q, kernel))
            model_analysis(Q, kernel, path, data, predictions, samples)
    return K, noise_std, X, Sigma, predictions

In [8]:
def model_eval(diagnostic, path, Q, kernel, data, predictions):
    elbo = diagnostic.loc[:, 'ELBO'].values[-1]
    R2_skl = skl.metrics.r2_score(data, predictions)
    R2_az = az.r2_score(data, predictions)
    ind = ['kernel', 'Q', 'ELBO', 'R2_skl', 'R2_az']
    num = np.random.random_integers(10000)
    mod_res = pd.DataFrame({num: (kernel, Q, elbo, R2_skl, R2_az)}, index=ind).T
    mod_res.to_csv(path+'eval_Q{}_kernel{}.csv'.format(Q, kernel))
    return mod_res

In [9]:
def hist_plotter(variable_mean, variable_name, path, Q, kernel):
    flatten = lambda l: [item for sublist in l for item in sublist]
    variable_np = np.asarray(variable_mean)
    np.delete(variable_np, 0, 1)
    np.delete(variable_np, 0, 1)

    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(1,1,1)
    ax = sns.distplot(variable_mean)
    ax.set_xlabel('values')
    ax.set_ylabel('value counts')
    ax.set_title(variable_name + " histogram")

    plt.savefig(path+'{}-hist_Q{}_kernel{}.png'.format(variable_name, Q, kernel))
    plt.close()

In [10]:
def quick_comp_plot(predictions, data, Q, path, kernel):
    fig = plt.figure(figsize=(40,20))
    x1, y1 = np.linspace(-0.1, 0.4, num=1000), np.linspace(-0.1, 0.4, num=1000)
    for n in range(11):
        sns.jointplot(x=predictions.iloc[n,:], y=data.iloc[n,:], data=predictions, kind="reg", truncate=False)
        plt.title(f'Returns of Stock {n}, Data vs Prediction')
        plt.legend(['Prediction vs Observation', 'Ideal'])
        plt.xlim(-0.1, 0.4)
        plt.ylim(-0.1, 0.4)
        plt.plot(x1, y1)
        plt.xlabel('Predictions')
        plt.ylabel('Observations/Data')
        plt.savefig(path+'Q{}_kernel{}_stock{}.png'.format(Q, kernel, n))
        plt.close()
        
        sns.jointplot(x=predictions.iloc[n,:], y=data.iloc[n,:], data=predictions, kind="kde", truncate=False)
        plt.title('Returns of Stock {}, Data vs Prediction'.format(n))
        plt.legend(['Prediction vs Observation', 'Ideal'])
        plt.xlim(-0.1, 0.4)
        plt.ylim(-0.1, 0.4)
        plt.plot(x1, y1)
        plt.xlabel('Predictions')
        plt.ylabel('Observations/Data')
        plt.savefig(path+'Q{}_kernel{}_stock{}.png'.format(Q, kernel, n))
        plt.close()

In [11]:
def StanModel_cache(model_code, model_name):
    cache_fn = f'cached-model-{model_name}.pkl'
    try:
        sm = pickle.load(open(cache_fn, 'rb'))
    except:
        sm = pystan.StanModel(model_code=model_code)
        with open(cache_fn, 'wb') as f:
            pickle.dump(sm, f)
    else:
        print("Using cached StanModel")
    return sm

In [12]:
def create_folders(model_name, time_period_name, ticker_list_name):
    if not os.path.exists(f'{model_name}/{time_period_name}-{ticker_list_name}/diagnostics/'):
        os.makedirs(f'{model_name}/{time_period_name}-{ticker_list_name}/diagnostics/')
    if not os.path.exists(f'{model_name}/{time_period_name}-{ticker_list_name}/samples/'):
        os.makedirs(f'{model_name}/{time_period_name}-{ticker_list_name}/samples/')

In [13]:
def model_analysis(Q, kernel, path, data, predictions, samples):
    model = LinearRegression()
    comp_ = []
    data = pd.DataFrame(data)
    diagnostic = pd.read_csv(path+'diagnostics/d_Q{}_kernel{}.csv'.format(Q, kernel), header=20)
    for n in range(1,N):
        a = predictions.iloc[n,:].values.reshape(-1,1)
        b = data.iloc[n-1,:]
        try:
            fit, R_2, intercept = model.fit(a,b), model.score(a,b), model.intercept_
        except ValueError:
            a, b = pd.DataFrame(a), pd.DataFrame(b)
            fit, R_2, intercept = model.fit(a,b), model.score(a,b), model.intercept_
        slope, = model.coef_
        comp__ = collections.OrderedDict()
        comp__ = {'R_2_fit':R_2, 'slope_fit':slope, 'intercept_fit':intercept}    
        comp_.append(comp__)
    comp = pd.DataFrame(comp_)
    comp.index.name='N'
    comp.to_csv(path+'diagnostics/comparison_Q{}_kernel{}.csv'.format(Q, kernel))
    model_eval(diagnostic, path, Q, kernel, data, predictions)
    hist_plotter(predictions, 'Y_hat', path, Q, kernel)
    quick_comp_plot(predictions, data, Q, path, kernel)

    #bfmi.to_csv('diagnostics/bfmi_Q{}_kernel{}.csv'.format(Q, kernel))
    #psis.to_csv('diagnostics/psis_Q{}_kernel{}.csv'.format(Q, kernel))
    return comp, print('Done with Q {} kernel {}.'.format(Q, kernel))

In [14]:
#Qs = [1,2,3,4,5,6]
#kernels = [1,2,3,4,5]
#returns = pd.read_csv('https://raw.githubusercontent.com/Kloppel/Thesis/master/resources/n120-d754-returns.csv', index_col='Date', parse_dates=['Date']).iloc[2:, :]
#Y = returns.T.values
#N, D = Y.shape
#data = Y

## GPLVM ##
#path = 'gplvm_finance/2010-trunc/'
#gplvm_predictions(Qs, kernels, path, data)

In [15]:
#Qs = [5,6]#1,2,3,4,5,6]
#kernels = [1,2,3,4,5]
#returns = pd.read_csv('https://raw.githubusercontent.com/Kloppel/Thesis/master/resources/n120-d754-returns.csv', index_col='Date', parse_dates=['Date']).iloc[2:, :]
#Y = returns.T.values
#N, D = Y.shape
#data = Y

## STUDT ##
#path = 'student-t/2010-trunc/'
#studt_predictions(Qs, kernels, path, data)

In [19]:
Qs = [1,2,3,4,5,6]
kernels = [1,2,3,4,5]
returns = pd.read_csv('https://raw.githubusercontent.com/Kloppel/Thesis/master/resources/n10-d501-returns.csv', index_col='Date', parse_dates=['Date']).iloc[2:, :]
Y = returns.T.values
N, D = Y.shape
data = Y

## VOLA, size largest ##
path = 'gplvm_vola/2010-N10-D501/'
vola_predictions(Qs, kernels, path, data)

ValueError: cannot reshape array of size 50100 into shape (499,10,10)

In [16]:
Qs = [1,2,3,4,5,6]
kernels = [1,2,3,4,5]
returns = pd.read_csv('https://raw.githubusercontent.com/Kloppel/Thesis/master/resources/n30-d122-returns.csv', index_col='Date', parse_dates=['Date']).iloc[2:, :]
Y = returns.T.values
N, D = Y.shape
data = Y

## VOLA, size like TIME ##
path = 'gplvm_vola/2010-vola/'
vola_predictions(Qs, kernels, path, data)

HTTPError: HTTP Error 404: Not Found